In [1]:
import os

In [2]:
import torch.nn as nn

class CommandClassifier(nn.Module):
    def __init__(self, input_dim=768, num_classes=35):
        super(CommandClassifier, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.classifier(x)

In [3]:
import torch
from transformers import Wav2Vec2Model
# from command_classifier import CommandClassifier

def load_model(model_path="trained_model/pytorch_model.bin"):
    base = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")
    head = CommandClassifier()
    # head.load_state_dict(torch.load(model_path, map_location=torch.device("cpu")))
    head.eval()
    return base, head

In [5]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base", is_tokenizer=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [13]:
import torch
from datasets import load_dataset
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
# from model_loader import load_model

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Load model
base, head = load_model()
# base.to(device)
# head.to(device)

# Load dataset
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
dataset = load_dataset("speech_commands", "v0.02", split="test[:1%]")

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/12.1k [00:00<?, ?B/s]

speech_commands.py:   0%|          | 0.00/7.31k [00:00<?, ?B/s]

The repository for speech_commands contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/speech_commands.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/84848 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9982 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4890 [00:00<?, ? examples/s]

In [16]:
# convert_to_onnx("facebook/wav2vec2-base")
# model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base")
base, head = load_model("facebook/wav2vec2-base")

In [8]:
import torchaudio
import numpy as np
import soundfile as sf
import onnxruntime as ort

In [14]:
ort_session = ort.InferenceSession("test.onnx")

In [46]:
import time
def inference(file):
    num_trials = 100  # Number of trials
    latencies = []
    # speech_array, sr = sf.read(file)
    features = processor(dataset[0]['audio']['array'], sampling_rate=16000, return_tensors="pt")
    input_values = features.input_values
    for _ in range(num_trials):
        start_time = time.time()
        if "onnx" in file:
          ort_session.run(None, {ort_session.get_inputs()[0].name: input_values.numpy()})[0]
        else:
          features = base(input_values).last_hidden_state.mean(dim=1)
          logits = head(features)
          pred = torch.argmax(logits, dim=1).item()
        latencies.append(time.time() - start_time)

    print(f"Inference Latency (single sample, median): {np.percentile(latencies, 50) * 1000:.2f} ms")
    print(f"Inference Latency (single sample, 95th percentile): {np.percentile(latencies, 95) * 1000:.2f} ms")
    print(f"Inference Latency (single sample, 99th percentile): {np.percentile(latencies, 99) * 1000:.2f} ms")
    print(f"Inference Throughput (single sample): {num_trials/np.sum(latencies):.2f} FPS")


In [47]:
inference("base, head")

Inference Latency (single sample, median): 333.91 ms
Inference Latency (single sample, 95th percentile): 547.06 ms
Inference Latency (single sample, 99th percentile): 552.70 ms
Inference Throughput (single sample): 2.72 FPS


In [45]:
inference("test.onnx")

Inference Latency (single sample, median): 282.94 ms
Inference Latency (single sample, 95th percentile): 454.40 ms
Inference Latency (single sample, 99th percentile): 602.42 ms
Inference Throughput (single sample): 3.14 FPS


In [48]:
inference("test_optimized.onnx")

Inference Latency (single sample, median): 280.93 ms
Inference Latency (single sample, 95th percentile): 444.25 ms
Inference Latency (single sample, 99th percentile): 452.29 ms
Inference Throughput (single sample): 3.25 FPS


In [63]:
inference("test_quantized_dynamic.onnx")

Inference Latency (single sample, median): 286.82 ms
Inference Latency (single sample, 95th percentile): 463.72 ms
Inference Latency (single sample, 99th percentile): 470.48 ms
Inference Throughput (single sample): 3.20 FPS


In [50]:
onnx_model_path = "test.onnx"
optimized_model_path = "test_optimized.onnx"

session_options = ort.SessionOptions()
session_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_EXTENDED # apply graph optimizations
session_options.optimized_model_filepath = optimized_model_path

ort_session = ort.InferenceSession(onnx_model_path, sess_options=session_options, providers=['CPUExecutionProvider'])

In [49]:
import neural_compressor
from neural_compressor import quantization


In [54]:
# Load ONNX model into Intel Neural Compressor
model_path = "test.onnx"
fp32_model = neural_compressor.model.onnx_model.ONNXModel(model_path)

# Configure the quantizer
config_ptq = neural_compressor.PostTrainingQuantConfig(
    approach="dynamic"
)

# Fit the quantized model
q_model = quantization.fit(
    model=fp32_model,
    conf=config_ptq
)

# Save quantized model
q_model.save_model_to_file("test_quantized_dynamic.onnx")


2025-05-11 16:30:57 [INFO] Start auto tuning.
2025-05-11 16:30:57 [INFO] Quantize model without tuning!
2025-05-11 16:30:57 [INFO] Quantize the model with default configuration without evaluating the model.                To perform the tuning process, please either provide an eval_func or provide an                    eval_dataloader an eval_metric.
2025-05-11 16:30:57 [INFO] Adaptor has 5 recipes.
2025-05-11 16:30:57 [INFO] 0 recipes specified by user.
2025-05-11 16:30:57 [INFO] 3 recipes require future tuning.
2025-05-11 16:30:57 [INFO] *** Initialize auto tuning
2025-05-11 16:30:57 [INFO] {
2025-05-11 16:30:57 [INFO]     'PostTrainingQuantConfig': {
2025-05-11 16:30:57 [INFO]         'AccuracyCriterion': {
2025-05-11 16:30:57 [INFO]             'criterion': 'relative',
2025-05-11 16:30:57 [INFO]             'higher_is_better': True,
2025-05-11 16:30:57 [INFO]             'tolerable_loss': 0.01,
2025-05-11 16:30:57 [INFO]             'absolute': None,
2025-05-11 16:30:57 [INFO]     

In [60]:
onnx_model_path = "test_quantized_dynamic.onnx"
model_size = os.path.getsize(onnx_model_path)
print(f"Model Size on Disk: {model_size/ (1e6) :.2f} MB")

Model Size on Disk: 95.46 MB


In [62]:
onnx_model_path = "test_quantized_dynamic.onnx"
ort_session = ort.InferenceSession(onnx_model_path, providers=['CPUExecutionProvider'])
